In [1]:
#
import duckdb
import os
import time
import pandas as pd
import fastparquet
import pyarrow as pa
import pyarrow.parquet as pq

In [27]:
con = duckdb.connect("subway-trips.db")

In [19]:
df = con.sql("SELECT * FROM trips LIMIT 5").df()

print(df)

   month day_of_week  hour_of_day    origin_station_complex_name  \
0      1      Monday            1               Canal St (A,C,E)   
1      1      Monday            1                    65 St (M,R)   
2      1      Monday            1                   Astor Pl (6)   
3      1      Monday            1  161 St-Yankee Stadium (B,D,4)   
4      1      Monday            1       Myrtle-Wyckoff Avs (L,M)   

  destination_station_complex_name  estimated_average_ridership  
0         Canal St (J,N,Q,R,W,Z,6)                       0.8476  
1                Junction Blvd (7)                       0.2000  
2       Clinton-Washington Avs (G)                       0.2400  
3                     Bay Pkwy (N)                       0.2000  
4                    Forest Av (M)                       0.6118  


In [12]:
columns_to_drop = ['year', 'timestamp','origin_station_complex_id', 'destination_station_complex_id', 'origin_latitude', 'destination_latitude', 'origin_longitude', 'destination_longitude', 'origin_point', 'destination_point' ]  # Replace with your actual column names

In [13]:
for column in columns_to_drop:
    con.execute(f"ALTER TABLE trips DROP COLUMN {column};")
    print(f"Column {column} dropped")

Column year dropped
Column timestamp dropped
Column origin_station_complex_id dropped
Column destination_station_complex_id dropped
Column origin_latitude dropped
Column destination_latitude dropped
Column origin_longitude dropped
Column destination_longitude dropped
Column origin_point dropped
Column destination_point dropped


In [14]:
result = con.execute("SELECT * FROM trips LIMIT 5;").fetchdf()
print(result)

   month day_of_week  hour_of_day    origin_station_complex_name  \
0      1      Monday            1               Canal St (A,C,E)   
1      1      Monday            1                    65 St (M,R)   
2      1      Monday            1                   Astor Pl (6)   
3      1      Monday            1  161 St-Yankee Stadium (B,D,4)   
4      1      Monday            1       Myrtle-Wyckoff Avs (L,M)   

  destination_station_complex_name  estimated_average_ridership  
0         Canal St (J,N,Q,R,W,Z,6)                       0.8476  
1                Junction Blvd (7)                       0.2000  
2       Clinton-Washington Avs (G)                       0.2400  
3                     Bay Pkwy (N)                       0.2000  
4                    Forest Av (M)                       0.6118  


In [31]:
# convert to parquet

# Specify the output Parquet file
parquet_file = 'subway-trips.parquet'

# Define the chunk size (number of rows per chunk)
chunk_size = 1000000  # Adjust based on your memory capacity and requirements

parquet_writer = None

# Start writing to the Parquet file
with open(parquet_file, 'wb') as f:
    # Use DuckDB's INSERT INTO syntax to write data in chunks
    # This will create the Parquet file in chunks, preventing memory overflow
    offset = 0
    while True:
        # Fetch the next chunk of data from the trips table
        start_time = time.time()
        df_chunk = con.execute(f"SELECT * FROM trips LIMIT {chunk_size} OFFSET {offset}").fetchdf()
        rows = df_chunk.shape[0]

        # Write the chunk to the Parquet file
        table = pa.Table.from_pandas(df_chunk)
        
        # Write the first chunk, then append subsequent chunks
        if parquet_writer is None:
            # Initialize the writer with the schema of the first chunk
            parquet_writer = pq.ParquetWriter(parquet_file, table.schema, compression='gzip')
        
        # Write the table to the Parquet file
        parquet_writer.write_table(table)
        
        if rows < chunk_size:
            print(f"Finished processing all data.")
            break
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Processed {offset + rows} rows. Elapsed time is {elapsed_time:.4f} seconds")

        # Move the offset forward
        offset += chunk_size
        
    # Close the ParquetWriter
    if parquet_writer:
        parquet_writer.close()



Processed 1000000 rows. Elapsed time is 1.5495 seconds
Processed 2000000 rows. Elapsed time is 1.4404 seconds
Processed 3000000 rows. Elapsed time is 0.9691 seconds
Processed 4000000 rows. Elapsed time is 1.0603 seconds
Processed 5000000 rows. Elapsed time is 0.9530 seconds
Processed 6000000 rows. Elapsed time is 0.9802 seconds
Processed 7000000 rows. Elapsed time is 0.9654 seconds
Processed 8000000 rows. Elapsed time is 0.9686 seconds
Processed 9000000 rows. Elapsed time is 0.9228 seconds
Processed 10000000 rows. Elapsed time is 1.0160 seconds
Processed 11000000 rows. Elapsed time is 1.0334 seconds
Processed 12000000 rows. Elapsed time is 1.0170 seconds
Processed 13000000 rows. Elapsed time is 0.9943 seconds
Processed 14000000 rows. Elapsed time is 1.0084 seconds
Processed 15000000 rows. Elapsed time is 1.0088 seconds
Processed 16000000 rows. Elapsed time is 0.9928 seconds
Processed 17000000 rows. Elapsed time is 1.0058 seconds
Processed 18000000 rows. Elapsed time is 1.0533 seconds
P

In [14]:
file_size = os.path.getsize('subway-trips.parquet')

# Convert the size to megabytes (MB) for easier readability
file_size_mb = file_size / (1024 * 1024)

print(f"Size of subway-trips.parquet: {file_size_mb:.2f} MB")

# Convert the size to gigabytes (GB)
file_size_gb = file_size / (1024 ** 3)  

print(f"Size of subway-trips.parquet: {file_size_gb:.2f} GB")

Size of subway-trips.parquet: 499.72 MB
Size of subway-trips.parquet: 0.49 GB


In [7]:
con = duckdb.connect()

In [3]:
con.sql(f"SELECT COUNT(*) FROM 'subway-trips.parquet'").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    115731896 │
└──────────────┘



In [8]:
con2 = duckdb.connect("subway-trips.db")

In [9]:
con2.sql(f"SELECT COUNT(*) FROM trips").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    115731896 │
└──────────────┘



In [13]:
# Close the database connection
con.close()

In [12]:
con2.close()